#CLassificazione con rappresentazione TFIDF

Compito di classificazione del testo rispetto alle seguenti categorie:


*   Automotive
*   Patio, Lawn and Garden
*   Office Products
*   Toys_&_Games



Mount Drive

In [ ]:
import random
random.seed(2020)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


Import librerie

In [ ]:
!pip install contractions

     |████████████████████████████████| 245kB 12.7MB/s 
     |████████████████████████████████| 317kB 31.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81708 sha256=521f45edcc0466ba59472144f4bc6f1b85c3e8ba3883f7cc12a9bf4aea5af2ac
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 1.9MB 8.7MB/s 


In [ ]:
import gzip
import numpy as np
import json
import pandas as pd
from sklearn import utils
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, treebank
import contractions
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
from spellchecker import SpellChecker
from itertools import chain
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install ipython-autotime
!pip install ipdb
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=e9069085f22b0c834cdb05d4b281b9f1ed6a9cf03e35476a2a0f59d1775249ce
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
  Created wheel for ipdb: filename=ipdb-0.13.3-cp36-none-any.whl size=10847 sha256=056ee317b32078e81ed40e479a7cb24a60df5877e2c1ac272f065c7b5258025e
  Stored in directory: /root/.cache/pip/wheels/75/00/30/4169bcc3643f0cf946dcf37af1b71364b390c4df91da02b03c
Successfully built ipdb


In [ ]:
pd.set_option('display.max_colwidth', -1)

time: 2.24 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


##Caricamento dati

In [ ]:
collection_stem = pd.read_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/post_stopwords_stem_filtro.csv")
collection_lemma = pd.read_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/post_stopwords_lemma_filtro.csv")

time: 18.4 s


In [ ]:
print(len(collection_stem), len(collection_lemma))

111860 111860
time: 1.22 ms


In [ ]:
collection_stem.drop(columns=["Unnamed: 0"], inplace = True)
collection_lemma.drop(columns=["Unnamed: 0"], inplace = True)

time: 73 ms


In [ ]:
collection_lemma = collection_lemma.sample(frac = 1, random_state = 2020)
collection_stem = collection_stem.sample(frac = 1, random_state = 2020)

time: 118 ms


In [ ]:
def tostring(tokens):
  tokens = str(tokens)
  tokens = tokens.replace("[", "")
  tokens = tokens.replace("]", "")
  tokens = tokens.replace("'", "")
  tokens = tokens.replace(",", "")
  return tokens

time: 2.98 ms


##Funzione di CV

In [ ]:
!pip install XGBoost

time: 1.95 s


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score

def Cross_Validation(features, target, classificatore):
  kfold = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
  cvscores = []

  i = 1
  for train, test in kfold.split(features, target):
    X_train, X_val = features[train], features[test]
    y_train, y_val = target.values[list(train)], target.values[list(test)]

    if str(type(classificatore)) == "<class 'keras.engine.sequential.Sequential'>":
      label_encoder = LabelEncoder()
      y_train = label_encoder.fit_transform(y_train)
      y_val = label_encoder.fit_transform(y_val)
      # fit model
      classificatore.fit(X_train, y_train)
      # predict
      pred = classificatore.predict_class(X_val)
    else:
      # fit model
      classificatore.fit(X_train, y_train)
      # predict
      pred = classificatore.predict(X_val)
    
    accuracy = accuracy_score(y_val, pred)
    print('Iterazione numero: ' + str(i))
    print('Accuracy attuale del modello di classificazione: ' + str(accuracy))

    cvscores.append(accuracy)
    i +=1

  print("Il punteggio medio ottenuto dalla cross-validation è " + str(np.mean(cvscores)) + " con una deviazione di " + str(np.std(cvscores)))
  return cvscores


time: 18.6 ms


#Creazione matrice TFIDF e Classificazione

##Lemma No-filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 7.02 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_lemma = tfidf.fit_transform(map(tostring, collection_lemma["lemmanosw"]))
tfidfmatrix_lemma.shape

(111860, 77440)

time: 6.85 s


In [ ]:
features_lemma = tfidfmatrix_lemma
target_lemma = collection_lemma.target

time: 1.72 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8517343107455748
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8558465939567317
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8517343107455748
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8493652780261041
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8532093688539245
Il punteggio medio ottenuto dalla cross-validation è 0.8523779724655821 con una deviazione di 0.0021273214899812814


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,TFIDF,"[0.8517343107455748, 0.8558465939567317, 0.8517343107455748, 0.8493652780261041, 0.8532093688539245]"


time: 40min 56s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8489182907205436
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8524941891650277
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8476220275344181
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8480243161094225
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8507509386733417
Il punteggio medio ottenuto dalla cross-validation è 0.8495619524405507 con una deviazione di 0.0018195403270155844


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,TFIDF,"[0.8489182907205436, 0.8524941891650277, 0.8476220275344181, 0.8480243161094225, 0.8507509386733417]"


time: 39min 37s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.9225370999463616
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.9239227605935991
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.9258001072769534
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.9258001072769534
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.9230287859824781
Il punteggio medio ottenuto dalla cross-validation è 0.9242177722152691 con una deviazione di 0.0013662318902025814


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,TFIDF,"[0.9225370999463616, 0.9239227605935991, 0.9258001072769534, 0.9258001072769534, 0.9230287859824781]"


time: 12 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8618809225817987
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8641605578401573
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8586626139817629
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8623279098873592
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8591989987484355
Il punteggio medio ottenuto dalla cross-validation è 0.8612462006079028 con una deviazione di 0.00204608896848489


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,TFIDF,"[0.8618809225817987, 0.8641605578401573, 0.8586626139817629, 0.8623279098873592, 0.8591989987484355]"


time: 8min 57s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_tfidf_lemma_nofiltro.csv")

time: 239 ms


##Lemma Filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 9.52 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_lemma_filtrata = tfidf.fit_transform(map(tostring, collection_lemma["lemmanosw2"]))
tfidfmatrix_lemma_filtrata.shape

(111860, 34393)

time: 4.63 s


In [ ]:
features_lemma = tfidfmatrix_lemma_filtrata
target_lemma = collection_lemma.target

time: 1.91 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.862238512426247
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8646075451457179
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.860495261934561
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.862193813695691
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8641158591096013
Il punteggio medio ottenuto dalla cross-validation è 0.8627301984623637 con una deviazione di 0.0014811408480047525


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,TFIDF,"[0.862238512426247, 0.8646075451457179, 0.860495261934561, 0.862193813695691, 0.8641158591096013]"


time: 32min 20s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8176738780618631
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8211156803146791
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8149025567673878
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8147237618451636
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8154836402646165
Il punteggio medio ottenuto dalla cross-validation è 0.8167799034507421 con una deviazione di 0.002409589124252391


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,TFIDF,"[0.8176738780618631, 0.8211156803146791, 0.8149025567673878, 0.8147237618451636, 0.8154836402646165]"


time: 25min 55s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.9138208474879314
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.9142678347934918
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.9146701233684963
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.91440193098516
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.9159216878240658
Il punteggio medio ottenuto dalla cross-validation è 0.9146164848918291 con una deviazione di 0.0007081046432438035


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,TFIDF,"[0.9138208474879314, 0.9142678347934918, 0.9146701233684963, 0.91440193098516, 0.9159216878240658]"


time: 9.51 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8314857858036832
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8387269801537637
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8345252994814947
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8301895226175576
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8361791525120686
Il punteggio medio ottenuto dalla cross-validation è 0.8342213481137135 con una deviazione di 0.0030972575792152855


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,TFIDF,"[0.8314857858036832, 0.8387269801537637, 0.8345252994814947, 0.8301895226175576, 0.8361791525120686]"


time: 5min


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_tfidf_lemma_filtro.csv")

time: 588 ms


##Stem No-filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 8.15 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_stem = tfidf.fit_transform(map(tostring, collection_stem["stemnosw"]))
tfidfmatrix_stem.shape

(111860, 60183)

time: 6.77 s


In [ ]:
features_stem = tfidfmatrix_stem
target_stem = collection_stem.target

time: 1.88 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8507509386733417
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8591543000178795
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8521365993205793
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8506168424816736
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8565170749150725
Il punteggio medio ottenuto dalla cross-validation è 0.8538351510817094 con una deviazione di 0.0034129463917076785


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,TFIDF,"[0.8507509386733417, 0.8591543000178795, 0.8521365993205793, 0.8506168424816736, 0.8565170749150725]"


time: 37min 11s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8534328625067048
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8567852672984088
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8515108170927945
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8524494904344717
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8541480421956017
Il punteggio medio ottenuto dalla cross-validation è 0.8536652959055964 con una deviazione di 0.001796733294389046


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,TFIDF,"[0.8534328625067048, 0.8567852672984088, 0.8515108170927945, 0.8524494904344717, 0.8541480421956017]"


time: 41min 2s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.9214643304130162
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.9249955301269444
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.9256213123547291
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.925487216163061
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.9253084212408368
Il punteggio medio ottenuto dalla cross-validation è 0.9245753620597176 con una deviazione di 0.0015695811415748096


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,TFIDF,"[0.9214643304130162, 0.9249955301269444, 0.9256213123547291, 0.925487216163061, 0.9253084212408368]"


time: 12.7 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8628642946540318
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8657250134096192
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8605846593956732
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8649204362596102
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8614339352762381
Il punteggio medio ottenuto dalla cross-validation è 0.8631056677990344 con una deviazione di 0.0019678816014136375


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,TFIDF,"[0.8628642946540318, 0.8657250134096192, 0.8605846593956732, 0.8649204362596102, 0.8614339352762381]"


time: 9min 25s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_tfidf_stem_nofiltro.csv")

time: 511 ms


##Stem Filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 6.03 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_stem_filtrata = tfidf.fit_transform(map(tostring, collection_stem["stemnosw2"]))
tfidfmatrix_stem_filtrata.shape

(111860, 27467)

time: 4.44 s


In [ ]:
features_stem = tfidfmatrix_stem_filtrata
target_stem = collection_stem.target

time: 967 µs


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8623726086179152
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8653674235651707
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8623279098873592
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8633112819595924
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8659932057929555
Il punteggio medio ottenuto dalla cross-validation è 0.8638744859645986 con una deviazione di 0.001528565881386838


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,TFIDF,"[0.8623726086179152, 0.8653674235651707, 0.8623279098873592, 0.8633112819595924, 0.8659932057929555]"


time: 30min 48s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8179867691757554
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8216967638119077
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8154389415340604
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8161541212229573
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8180314679063114
Il punteggio medio ottenuto dalla cross-validation è 0.8178616127301984 con una deviazione di 0.002169689457417826


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,TFIDF,"[0.8179867691757554, 0.8216967638119077, 0.8154389415340604, 0.8161541212229573, 0.8180314679063114]"


time: 25min 7s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.9121222957268014
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.9134632576434829
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.9146701233684963
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.9151171106740569
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.915295905596281
Il punteggio medio ottenuto dalla cross-validation è 0.9141337386018236 con una deviazione di 0.0011917053770415145


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,TFIDF,"[0.9121222957268014, 0.9134632576434829, 0.9146701233684963, 0.9151171106740569, 0.915295905596281]"


time: 9.86 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "TFIDF", "Punteggio_cv": Cross_Validation(features_stem, target_stem, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8118183443590202
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8221884498480243
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8215626676202396
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.804845342392276
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8123994278562489
Il punteggio medio ottenuto dalla cross-validation è 0.8145628464151619 con una deviazione di 0.006538958333715797


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,TFIDF,"[0.8118183443590202, 0.8221884498480243, 0.8215626676202396, 0.804845342392276, 0.8123994278562489]"


time: 5min 14s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_tfidf_stem_filtro.csv")

time: 452 ms
